In [52]:
from ZW_dataset import *

In [53]:
from config import DATA_DIRECTORY
std_classes = ["G", "T", "A", "C", "H", "a", "b", "1", "2", "-1", "-2", "E"]
datapath = DATA_DIRECTORY / "v21D0_m1.npy"
data = np.load(datapath, allow_pickle=True)

In [54]:
class contextGPTDataset(Dataset):
    def __init__(self, data, classes, block_size, training_type="standard"):
        self.base = data
        print("Designs in the dataset:", len(self.base))
        self.data = data
        self.classes = classes
        # Integer encoding
        self.data = string_to_equipment(self.data, self.classes)
        if training_type == "augmented":
            self.data = self.augment_data(self.data)
            print("Data augmented:", len(self.data) - len(self.base))
        for i in self.data:
            l = len(i)
            i.insert(0,l)
        self.data = torch.tensor([i + [11] * (block_size - len(i)) for i in self.data])
        # input output preparation
        self.data, self.labels = self.input_output_prep(self.data)
        # Output classes
        print("Input shape:", self.data.shape)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

    def input_output_prep(self, data):
        inputs = data[:, :-1]
        outputs = data[:, 1:]
        return inputs, outputs

    def augment_data(self, data):
        augmented = []
        for i in data:
            base = np.array(i)
            nognoe = base[1:-1]
            for j in range(1, len(nognoe)):
                new_rep = np.roll(nognoe, j, axis=0)
                augmented.append(
                    np.concatenate((base[0:1], new_rep, base[-1:]), axis=0).tolist()
                )
        return data + augmented